# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2021 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and Spark SQL, Moodle exercise</center>

# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
It will be the same language game dataset as in exercise08.

1. Change to exercise09 repository

2. Start docker <br>
```docker-compose up -d```

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. 

More specifically do the following:
- download the data      :<br> ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data       :<br> ```tar -jxvf confusion-2014-03-02.tbz2```

4. copy the data to docker :<br> ```docker cp confusion-2014-03-02/confusion-2014-03-02.json jupyter:/home/jovyan/work``` <br>
(Copying the data to docker needs to be done only once and it might take 1-2 minutes.)

## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: http://lars.yencken.org/datasets/languagegame/

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed before you start writing the query, and the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [1]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

## <center>1. Spark Dataframes</center>

Write queries for the same questions as last week, but this time using Spark Dataframes operations (the data loading will take a minute)

### 1.0. Data preprocessing

In [3]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()

In [4]:
#test it out
dataset.limit(3).show()

+--------------------+-------+----------+---------+--------------------+---------+
|             choices|country|      date|    guess|              sample|   target|
+--------------------+-------+----------+---------+--------------------+---------+
|[Maori, Mandarin,...|     AU|2013-08-19|Norwegian|48f9c924e0d98c959...|Norwegian|
|[Danish, Dinka, K...|     AU|2013-08-19|    Dinka|af5e8f27cef9e689a...|    Dinka|
|[German, Hungaria...|     AU|2013-08-19|  Turkish|509c36eb58dbce009...|   Samoan|
+--------------------+-------+----------+---------+--------------------+---------+



## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Maori.

In [12]:
start_exercise()

In [17]:
#Your code here
dataset.filter((dataset.target == 'Maori') & (dataset.target == dataset.guess)).count()

74810

In [14]:
finish_exercise()

This exercise took 3s


## Assignment 2
Return the number of distinct "target" languages.

In [18]:
start_exercise()

In [19]:
#Your code here
dataset.select('target').distinct().count()

78

In [20]:
finish_exercise()

This exercise took 21s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the bottom three games where the guessed language is incorrect (not equal to the target one) ordered by date (ascending), then by language (descending), then by country (ascending).

Return the sample IDs (i.e., the sample field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In [21]:
start_exercise()

In [26]:
#Your code here
dataset.select('sample').filter(dataset.target == dataset.guess).orderBy(["target", "country", "date"], ascending=[1, 1, 1]).limit(3).collect()

# 00b85faa8b878a14f8781be334deb137,efcd813daec1c836d9f030b30caa07ce,efcd813daec1c836d9f030b30caa07ce

[Row(sample='00b85faa8b878a14f8781be334deb137'),
 Row(sample='efcd813daec1c836d9f030b30caa07ce'),
 Row(sample='efcd813daec1c836d9f030b30caa07ce')]

In [24]:
finish_exercise()

This exercise took 299s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.


Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.
In order to help with the grading please add them in the following format: count1,count2,count3 (meaning separated with commas and without any spaces between them)

In [27]:
start_exercise()

In [28]:
#Your code here
dataset.groupBy([dataset.country, dataset.target]).count().orderBy(['count'], ascending=[0]).limit(3).collect()
# 112934,112007,110919

[Row(country='US', target='French', count=112934),
 Row(country='US', target='German', count=112007),
 Row(country='US', target='Spanish', count=110919)]

In [30]:
finish_exercise()

This exercise took 153s


## Assignment 5
Find the fraction (between 0 and 1) of games where (the answer was correct && the correct guess was the second choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3320)

In [31]:
start_exercise()

In [34]:
#Your code here
dataset.filter((dataset.target == dataset.guess) & (dataset.choices[1] == dataset.guess)).count() / dataset.count()


0.26400199040361877

In [33]:
finish_exercise()

This exercise took 114s


## Assignment 6
Sort the languages by increasing overall percentage of correct guesses and return the first three languages.

In [35]:
start_exercise()

In [41]:
#Your code here
d1 = dataset.filter(dataset.target == dataset.guess).groupBy(dataset.target).count().withColumnRenamed('count', 'c1')

d2 = dataset.groupBy(dataset.target).count().withColumnRenamed('count', 'c2')
                    
d3 = d1.join(d2, 'target')
                    
d4 = d3.select('*', (d3.c1 / d3.c2).alias('a'))
d4.orderBy([d4.a], ascending=[1]).limit(3).show()
# Kannada,Fijian,Shona

+-------+-----+------+-------------------+
| target|   c1|    c2|                  a|
+-------+-----+------+-------------------+
|Kannada|46923|119509|0.39263151729158474|
| Fijian|48081|115852|0.41502088872009113|
|  Shona|36186| 82478|0.43873517786561267|
+-------+-----+------+-------------------+



In [42]:
finish_exercise()

This exercise took 657s


## Assignment 7
Return the number of games played on the first day.

In [43]:
start_exercise()

In [53]:
#Your code here
from pyspark.sql.functions import min

dataset.select([min('date')]).show()
# 2013-08-19

dataset.filter(dataset.date == '2013-08-19').count()

+----------+
| min(date)|
+----------+
|2013-08-19|
+----------+



163

In [51]:
finish_exercise()

This exercise took 331s


## <center>2. Spark SQL</center>

Write Spark SQL queries for the same questions as earlier.

### 2.0. Data preprocessing

In [54]:
!pip install sparksql-magic

In [55]:
%load_ext sparksql_magic

In [56]:
path = "confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()
dataset.registerTempTable("dataset")

In [61]:
%%sparksql
-- test it out
SELECT *
FROM dataset
LIMIT 3

choices,country,date,guess,sample,target
"['Maori', 'Mandarin', 'Norwegian', 'Tongan']",AU,2013-08-19,Norwegian,48f9c924e0d98c959d8a6f1862b3ce9a,Norwegian
"['Danish', 'Dinka', 'Khmer', 'Lao']",AU,2013-08-19,Dinka,af5e8f27cef9e689a070b8814dcc02c3,Dinka
"['German', 'Hungarian', 'Samoan', 'Turkish']",AU,2013-08-19,Turkish,509c36eb58dbce009ccf93f375358d53,Samoan


## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Maori.

In [62]:
start_exercise()

In [65]:
%%sparksql
SELECT count(*) 
FROM dataset
WHERE target == guess AND target == "Maori" 

count(1)
74810


In [66]:
finish_exercise()

This exercise took 77s


## Assignment 2
Return the number of distinct "target" languages.

In [68]:
start_exercise()

In [70]:
%%sparksql
SELECT  count(distinct(target)) AS c
FROM dataset

c
78


In [71]:
finish_exercise()

This exercise took 96s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the bottom three games where the guessed language is incorrect (not equal to the target one) ordered by date (ascending), then by language (descending), then by country (ascending).

In [ ]:
start_exercise()

In [72]:
%%sparksql
SELECT  sample
FROM dataset
WHERE target == guess
ORDER BY target ASC, country ASC, date ASC
LIMIT 3

# 00b85faa8b878a14f8781be334deb137,efcd813daec1c836d9f030b30caa07ce,efcd813daec1c836d9f030b30caa07ce

sample
00b85faa8b878a14f8781be334deb137
efcd813daec1c836d9f030b30caa07ce
efcd813daec1c836d9f030b30caa07ce


In [73]:
finish_exercise()

This exercise took 312s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [ ]:
start_exercise()

In [75]:
%%sparksql
SELECT count(guess)
FROM dataset
GROUP BY country, target
ORDER BY count(guess) DESC
LIMIT 3

# 112934,112007,110919

count(guess)
112934
112007
110919


In [76]:
finish_exercise()

This exercise took 446s


## Assignment 5
Find the fraction (between 0 and 1) of games where (the answer was correct && the correct guess was the second choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3320)

In [77]:
start_exercise()

In [84]:
%%sparksql
WITH a AS(
SELECT count(*) AS a
FROM dataset
WHERE target == guess AND target == choices[1]),
b AS(
SELECT count(*) AS b
FROM dataset)
SELECT a.a / b.b FROM a, b

(CAST(a AS DOUBLE) / CAST(b AS DOUBLE))
0.26400199040361877


In [83]:
finish_exercise()

This exercise took 243s


In [82]:
4358996 / 16511224

0.26400199040361877

## Assignment 6
Sort the languages by increasing overall percentage of correct guesses and return the first three languages.

In [87]:
start_exercise()

In [88]:
%%sparksql
WITH a AS(
SELECT target, count(*) AS a
FROM dataset
WHERE target == guess 
GROUP BY target),
b AS(
SELECT target, count(*) AS b
FROM dataset
GROUP BY target)
SELECT a.target, a.a / b.b 
FROM a, b 
WHERE a.target = b.target
ORDER BY a.a / b.b ASC
LIMIT 3

# Kannada,Fijian,Shona

target,(CAST(a AS DOUBLE) / CAST(b AS DOUBLE))
Kannada,0.39263151729158474
Fijian,0.41502088872009113
Shona,0.43873517786561267


In [89]:
finish_exercise()

This exercise took 66s


## Assignment 7
Return the number of games played on the first day.

In [90]:
start_exercise()

In [94]:
%%sparksql
WITH a AS(
SELECT MIN(date) AS d 
    FROM dataset)
SELECT count(*) 
FROM dataset, a
WHERE dataset.date = a.d

count(1)
163


In [95]:
finish_exercise()

This exercise took 81s
